# **Install and Import Libraries**

> ##### **Add the OpenAI API key in config/secrets.env file as follows:**

> ###### **OPENAI_API_KEY = "<api_key>"**


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from dotenv import load_dotenv

# load config
load_dotenv("../config/config.env")

# load secrets
load_dotenv("../config/secrets.env")

from data_pipeline import *

# **1. Scrape Website**

In [ ]:
df = scrape_website()

In [ ]:
df

# **2. Download all PDFs from links**

In [ ]:
df = download_documents()

In [ ]:
df

# **3. Extract HTML from PDFs**

In [ ]:
convert_pdf_to_html()

# **4. Extract Meeting Metadata from PDF with LLM**

In [ ]:
df = extract_meeting_metadata()

In [ ]:
df

# **5. Extract Proposals and Decisions**

## still not working as expected

In [ ]:
df = extract_meeting_agenda()